# Esercizio 2


### Appunti

- Prendo le def
- calcolo i termini più frequenti
- faccio disambiguation
- ottengo synset piu frequenti - che saranno i possibili genus
- confronto 

*DEF di genus:*  Iperonimo più comune dell'oggetto che si pensa che l'interlocutore possa conoscere al meglio; Non sono elementi iper generici o iper specifici, sono ad un livello mediio che più o meno tutti conosciamo (es. mela, pera --> Frutto)

Il punto è non cofrontare tutte le definizioni (100K+) ma confrontarle solo con i genus

In [8]:
from string import punctuation
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from collections import Counter
from nltk.wsd import lesk

## Utilities

In [9]:
def word_sense_disambiguation(list_words, word):
    '''
    Using wordnet's lesk function for disambiguation,
    return the first synset
    '''
    right_synset = lesk(list_words, word)
    return right_synset

def remove_stop_words(row):
    stop_words = set(stopwords.words('english'))
    punctuation = [',', '.', ';', '!', '?', "'", "''", '"', "’", "’’", "`","``"]
    filtered_sentence = [w for w in row if not w.lower() in stop_words and w not in punctuation]
    return filtered_sentence

def calculate_frequency(row):
    '''
    Calculate the frequency of each word in the sentence
    '''
    freq_dict = {}
    row = remove_stop_words(word_tokenize(row))
    for word in row[1:]:
        if word.lower() not in freq_dict:
            freq_dict[word.lower()] = 1
        else:
            freq_dict[word.lower()] += 1
    return freq_dict

def freq_synset(list):
    '''
    Calculate the frequency of each synset in a list of synsets
    '''
    freq_dict = {}
    for word in list:
        if word not in freq_dict:
            freq_dict[word] = 1
        else:
            freq_dict[word] += 1
    return freq_dict

## Execution

### Print the most common word in the definitions with their synset

In [13]:
with open ('../res/def.csv', 'r') as f:
    for row in f:
        dict = calculate_frequency(row)
        synsets, genus_list = [], []
        count = Counter(dict)
        most_freq = count.most_common(5)
        #print(f'\n--> Starting word: *{row.split(",")[0]}*\n')
        
        for word in most_freq:
            # Check if there are ampty values and save the first synset
            if(len(wn.synsets(word[0])) > 0):
                #print(f"--> Word: {word} \n--> Synset: {wn.synsets(word[0])[0]}\n")
                synsets.append(wn.synsets(word[0])[0])
                
        # for each synset let find all hyponyms and hypernyms and then compare them
        for syn in synsets:
            genus_list.append(syn.hypernyms())
            # genus_list.append(syn.hyponyms())

        # list of list to list
        genus = [x for xs in genus_list for x in xs]
        count_genus = Counter(freq_synset(genus))
        print(f'{row.split(",")[0]}: {count_genus.most_common(5)}')

        

Emotion: [(Synset('state.n.02'), 1), (Synset('hominid.n.01'), 1), (Synset('awareness.n.01'), 1), (Synset('administrative_district.n.01'), 1)]
Person: [(Synset('hominid.n.01'), 1), (Synset('person.n.01'), 1), (Synset('causal_agent.n.01'), 1), (Synset('organism.n.01'), 1), (Synset('experience.n.02'), 1)]
Revenge: [(Synset('causal_agent.n.01'), 1), (Synset('organism.n.01'), 1), (Synset('state.n.02'), 1), (Synset('emotion.n.01'), 1), (Synset('act.n.02'), 1)]
Brick: [(Synset('physical_entity.n.01'), 1), (Synset('substance.n.01'), 1), (Synset('creating_from_raw_materials.n.01'), 1), (Synset('bodily_property.n.01'), 1)]
